# ESA사이트의 데이터를 활용하여 진행

### 1. 사이트에서 데이터를 직접 다운받아보자.

- ESA(European Space Agency) – Sentinel 위성
    - https://dataspace.copernicus.eu/
    - `L1C`: TOA 반사도
    - `L2A`: 대기보정 완료 (실무에서는 거의 L2A 사용)
- **사용법**
    1. search 탭 클릭
    2. data sources에서 위성 선택
        - **MSI** → Sentinel-2 위성에 탑재된 광학 **다중분광 센서**
        (→ 하나의 센서가 여러 파장대를 동시에 관측)
            - 총 13개의 밴드를 제공
            - 10m : B2, B3, B4, B8
            - 20m : B5, B6, B6, B8A, B11, B12
            - 60m : B1, B9, B10

        ❓ 하나의 센서인데 왜 밴드별 해상도가 다를까?

        - 짧은 파장은 상대적으로 센서가 많은 빛을 받음 → 10m 가능
        - 긴 파장 또는 대기 관련 밴드는 빛이 적게 들어오기 때문에 픽셀을 크게 해야 SNR(신호대잡음비) 확보 가능
            - SNR이 **높을수록** 센서가 측정값을 정확하게 읽음 → 픽셀 값 신뢰 ↑
            - B1 → B12로 갈수록 일반적으로 파장이 길어진다.

            ❓ 그렇다면 왜 B1은 낮은 해상도인 60m를 제공하는가?

            B1은 지표 분석용이 아니라 대기보정용 밴드이다.

            지표 분석용이 아니므로 고해상도가 불필요하다

    3. 기간 설정 후 Search 버튼 클릭
    4. 특정 지역에 해당하는 영상을 검색하고 싶다면?
    → 오른쪽 위의 폴리곤을 클릭 후 해당 영역을 선택하면 된다.

- 하나의 지역에 대하여 여러 위성 영상 다운받기
- 다운받은 .SAFE 파일 사용하기
    - QGIS 이용(일단 단순 시각화)
        - 실질적인 파일은 GRANULE/…/IMG_DATA 내부 경로에 존재
        - 원하는 .tif 파일을 선택하여 진행
    - 파이썬 이용(→ 아주 간단한 확인용)

        ```jsx
        with rasterio.open(tif_path) as src:
            # 밴드 읽기 (흑백/단일 밴드)
            band1 = src.read(1)
            print(band1.shape, band1.dtype)

            # 시각화
            plt.imshow(band1, cmap='gray')
            plt.title("JP2 Image - Band 1")
            plt.show()
        ```

## B04, B03, B02 --> 하나의 .TIF 파일

In [ ]:
# (시도) 3개의 R,G,B 파일 -->  하나의 RGB파일(.TIF)
import rasterio
import numpy as np

arrays = []
meta = None

for band in ['B02', 'B03', 'B04']:
    tif_path = f"Z:\\users\huiwon\\2026상반기_지리정보데이터_공부\\20260106_확장자 공부_샘플 데이터\S2B_MSIL2A_20240501T185919_N0510_R013_T10TEL_20240501T230849.SAFE\GRANULE\L2A_T10TEL_A037362_20240501T190902\IMG_DATA\R20m\T10TEL_20240501T185919_{band}_20m.jp2"

    with rasterio.open(tif_path) as src:
        arrays.append(src.read(1))
        if meta is None:
            meta = src.meta

stacked = np.stack(arrays, axis=0)

meta.update({
    "count": 3
})

with rasterio.open("data2/merged_3band.tif", "w", **meta) as dst:
    dst.write(stacked)

In [ ]:
# (시도) 3개의 R,G,B 파일 -->  하나의 RGB파일(.TIF) | 최대값이 255가 되도록

import rasterio
import numpy as np

arrays = []
meta = None

for band in ['B02', 'B03', 'B04']:
    tif_path = f"Z:\\users\huiwon\\2026상반기_지리정보데이터_공부\\20260106_확장자 공부_샘플 데이터\S2B_MSIL2A_20240501T185919_N0510_R013_T10TEL_20240501T230849.SAFE\GRANULE\L2A_T10TEL_A037362_20240501T190902\IMG_DATA\R20m\T10TEL_20240501T185919_{band}_20m.jp2"

    with rasterio.open(tif_path) as src:
        arrays.append(src.read(1))
        if meta is None:
            meta = src.meta

stacked = np.stack(arrays, axis=0)
scaled = np.clip(stacked / 10000 * 255, 0, 255).astype(np.uint8)

meta.update({
    "count": 3,
    "dtype": "uint8"
})

with rasterio.open("data2/merged_3band_scaled_2.tif", "w", **meta) as dst:
    dst.write(scaled)


In [ ]:
# ===== (최종) 히스토그램 Stretch + 감마 보정된 =====

import rasterio
import numpy as np

bands = ['B04','B03','B02']  # R,G,B
arrays = []
meta = None

for band in bands:
    tif_path = f"Z:\\users\huiwon\\2026상반기_지리정보데이터_공부\\20260106_확장자 공부_샘플 데이터\S2B_MSIL2A_20240501T185919_N0510_R013_T10TEL_20240501T230849.SAFE\GRANULE\L2A_T10TEL_A037362_20240501T190902\IMG_DATA\R20m\T10TEL_20240501T185919_{band}_20m.jp2"

    with rasterio.open(tif_path) as src:
        arrays.append(src.read(1))
        if meta is None:
            meta = src.meta

stacked = np.stack(arrays, axis=0)

# 히스토그램 Stretch (2~98% 백분위)
scaled = np.zeros_like(stacked, dtype=np.uint8)
for i in range(3):
    band = stacked[i]
    low, high = np.percentile(band, (2, 98))
    band_stretched = np.clip((band - low) / (high - low) * 255, 0, 255)

    # 감마 보정 (γ=1/2.2)
    gamma = 1/2.2
    band_gamma = np.power(band_stretched/255, gamma) * 255
    scaled[i] = band_gamma.astype(np.uint8)

# meta 업데이트
meta.update({"count":3,"dtype":"uint8"})

# 저장
out_path = "data2/L2A_RGB_stretched_2.tif"
with rasterio.open(out_path,"w",**meta) as dst:
    dst.write(scaled)

print("히스토그램 Stretch + 감마 보정된 RGB TIF 생성 완료")


In [ ]:
# (시도) 히스토그램 Stretch
import rasterio
import numpy as np
import os

bands = ['B04','B03','B02']  # R,G,B
arrays = []
meta = None

for band in bands:
    tif_path = f"Z:\\users\huiwon\\2026상반기_지리정보데이터_공부\\20260106_확장자 공부_샘플 데이터\S2B_MSIL2A_20240501T185919_N0510_R013_T10TEL_20240501T230849.SAFE\GRANULE\L2A_T10TEL_A037362_20240501T190902\IMG_DATA\R20m\T10TEL_20240501T185919_{band}_20m.jp2"
    with rasterio.open(tif_path) as src:
        arrays.append(src.read(1))
        if meta is None:
            meta = src.meta

stacked = np.stack(arrays, axis=0)

# 히스토그램 Stretch만 적용 (2~98% 백분위)
scaled = np.zeros_like(stacked, dtype=np.uint8)
for i in range(3):
    band = stacked[i]
    scaled[i] = np.clip(band / band.max() * 255, 0, 255).astype(np.uint8)


# meta 업데이트
meta.update({"count":3,"dtype":"uint8"})

# 저장
out_path = "data2/L2A_RGB_stretch_only.tif"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

with rasterio.open(out_path, "w", **meta) as dst:
    dst.write(scaled)

print("히스토그램 Stretch만 적용된 RGB TIF 생성 완료")


- 마지막 셀의 작업인 히스토그램 stretch와 gamma 보정이 적용된 코드가 제일 적합(시각화를 진행하기에)

## 다운받은 데이터를 이용하여 NDVI 시각화 진행

- input 데이터는 .jp2(jpeg2000)
- 따라서 자동으로 output을 저장할 때도 .jp2의 형태로 진행됨.
- 그러냐 **NDVI**의 경우에는 **float32**를 이용해서 계산을 해야 함.
- RGB 파일을 만들 때 처럼 unit8로 계산을 하면 -1, 1 사이의 값을 가지지 못함
- 해결법: meta의 driver 정보에 저장형식 지정→GTIFF

In [ ]:
# (시도) NDVI float32가 파일 생성
import rasterio
import numpy as np

# 밴드 경로 설정
tif_folder_path = f"Z:\\users\huiwon\\2026상반기_지리정보데이터_공부\\20260106_확장자 공부_샘플 데이터\S2B_MSIL2A_20240501T185919_N0510_R013_T10TEL_20240501T230849.SAFE\GRANULE\L2A_T10TEL_A037362_20240501T190902\IMG_DATA\R10m"

red_path = f"{tif_folder_path}\\T10TEL_20240501T185919_B04_10m.jp2"
nir_path = f"{tif_folder_path}\\T10TEL_20240501T185919_B08_10m.jp2"

with rasterio.open(red_path) as red_src:
    red = red_src.read(1).astype(float)

with rasterio.open(nir_path) as nir_src:
    nir = nir_src.read(1).astype(float)
    meta = nir_src.meta  # NDVI 저장용

# NDVI 계산
ndvi = (nir - red) / (nir + red)
ndvi = np.clip(ndvi, -1, 1)  # NDVI 범위 제한

# meta 업데이트
meta.update({"dtype": "float32", "count": 1})

# NDVI TIF 저장
out_path = "data2/NDVI.tif"
with rasterio.open(out_path, "w", driver="GTiff", **meta) as dst:
    dst.write(ndvi.astype(np.float32), 1)

print("NDVI TIF 생성 완료")


In [ ]:
# ===== (최종) NDVI float32가 파일 생성 =====

import rasterio
import numpy as np
import os

# 밴드 경로
tif_folder_path = r"Z:\users\huiwon\2026상반기_지리정보데이터_공부\20260106_확장자 공부_샘플 데이터\S2B_MSIL2A_20240501T185919_N0510_R013_T10TEL_20240501T230849.SAFE\GRANULE\L2A_T10TEL_A037362_20240501T190902\IMG_DATA\R10m"

red_path = os.path.join(tif_folder_path, "T10TEL_20240501T185919_B04_10m.jp2")
nir_path = os.path.join(tif_folder_path, "T10TEL_20240501T185919_B08_10m.jp2")

# 밴드 읽기
with rasterio.open(red_path) as src:
    red = src.read(1).astype(float)

with rasterio.open(nir_path) as src:
    nir = src.read(1).astype(float)
    meta = src.meta.copy()  # meta 복사

# NDVI 계산
ndvi = (nir - red) / (nir + red)
ndvi = np.clip(ndvi, -1, 1)

# meta 업데이트 (driver 포함)
meta.update({
    "driver": "GTiff",  # GeoTIFF로 저장
    "dtype": "float32",
    "count": 1
})

# NDVI 저장
out_path = "data2/NDVI_float32.tif"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

with rasterio.open(out_path, "w", **meta) as dst:
    dst.write(ndvi.astype(np.float32), 1)

print("NDVI float32 GeoTIFF 생성 완료")

# USGS 사이트의 데이터를 활용하여 진행

In [12]:
# (시도) gee의 데이터를 시각화해보기
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='ee-gmldnjs295')

mosaic = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
    .filterDate('2019-06-01', '2019-06-16')
    .mosaic()
    .set('SENSOR_ID', 'OLI_TIRS')
)

# Cloud score the mosaic and display the result.
scored_mosaic = ee.Algorithms.Landsat.simpleCloudScore(mosaic)
m = geemap.Map()
m.add_layer(
    scored_mosaic,
    {'bands': ['B4', 'B3', 'B2'], 'max': 0.4},
    'TOA mosaic',
)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [28]:
# ===== (최종) ee를 이용하여 진행되는 작업을 ee없이 진행해보기 + cv2=====

# =========================
# Landsat SR RGB 미리보기(OpenCV) + QGIS용 GeoTIFF 저장
# =========================

import rasterio
import numpy as np
import cv2

print("STEP 0. 라이브러리 로드 완료")

# -------------------------
# 1. 파일 경로
# -------------------------
DATA_DIR = "./data3"

B2_PATH = f"{DATA_DIR}/LC09_L2SP_116035_20251230_20260102_02_T1_SR_B2.TIF"
B3_PATH = f"{DATA_DIR}/LC09_L2SP_116035_20251230_20260102_02_T1_SR_B3.TIF"
B4_PATH = f"{DATA_DIR}/LC09_L2SP_116035_20251230_20260102_02_T1_SR_B4.TIF"
QA_PATH = f"{DATA_DIR}/LC09_L2SP_116035_20251230_20260102_02_T1_QA_PIXEL.TIF"

DOWNSAMPLE = 10   # OpenCV 미리보기용
RGB_MAX = 0.4     # 스케일 최대값

print("STEP 1. 파일 경로 설정 완료")

# -------------------------
# 2. 밴드 로드 함수 (SR scaling)
# -------------------------
def load_band_sr(path):
    with rasterio.open(path) as src:
        data = src.read(1).astype(np.float32)
        nodata = src.nodata
        if nodata is not None:
            data[data == nodata] = np.nan
    # Landsat SR 스케일
    data = data * 2.75e-05 - 0.2
    return data

print("STEP 2. 밴드 로드 함수 정의 완료")

# -------------------------
# 3. QA_PIXEL 기반 구름 마스크
# -------------------------
with rasterio.open(QA_PATH) as src:
    qa = src.read(1)

cloud = ((qa & (1 << 3)) != 0) | ((qa & (1 << 4)) != 0)
mask = ~cloud
print(f"STEP 3. 구름 마스크 생성 완료 | cloud pixel 수 = {np.sum(cloud)}")

# -------------------------
# 4. RGB 밴드 로드 + 마스크
# -------------------------
r = load_band_sr(B4_PATH)
g = load_band_sr(B3_PATH)
b = load_band_sr(B2_PATH)

r[~mask] = np.nan
g[~mask] = np.nan
b[~mask] = np.nan
print("STEP 4. RGB 밴드 로드 및 구름 마스크 적용 완료")

# -------------------------
# 5. OpenCV 미리보기용 다운샘플 + 퍼센타일 스트레치
# -------------------------
def stretch_2_98(band):
    valid = np.isfinite(band)
    p2, p98 = np.percentile(band[valid], (2, 98))
    return np.clip((band - p2) / (p98 - p2), 0, 1)

r_ds = stretch_2_98(r[::DOWNSAMPLE, ::DOWNSAMPLE])
g_ds = stretch_2_98(g[::DOWNSAMPLE, ::DOWNSAMPLE])
b_ds = stretch_2_98(b[::DOWNSAMPLE, ::DOWNSAMPLE])

rgb_preview = np.dstack([r_ds, g_ds, b_ds])
rgb8 = (rgb_preview * 255).astype(np.uint8)

# OpenCV는 BGR 순서
bgr = cv2.cvtColor(rgb8, cv2.COLOR_RGB2BGR)
# cv2.imwrite("landsat_rgb_preview.png", bgr)  # PNG

cv2.imshow("Landsat RGB Preview", bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()
print("STEP 5. OpenCV 미리보기 완료")

# -------------------------
# 6. QGIS용 GeoTIFF 저장 (원본 해상도 + 퍼센타일 스트레치)
# -------------------------
r_s = stretch_2_98(r)
g_s = stretch_2_98(g)
b_s = stretch_2_98(b)

rgb_qgis = np.dstack([r_s, g_s, b_s])
rgb_uint8 = (rgb_qgis * 255).astype(np.uint8)

# 메타데이터 가져오기
with rasterio.open(B4_PATH) as src:
    meta = src.meta.copy()

meta.update({
    "count": 3,
    "dtype": "uint8",
    "compress": "lzw",
    "nodata": 255
})

OUT_PATH = "data3/landsat_rgb_qgis.tif"
with rasterio.open(OUT_PATH, "w", **meta) as dst:
    dst.write(rgb_uint8[:, :, 0], 1)  # R
    dst.write(rgb_uint8[:, :, 1], 2)  # G
    dst.write(rgb_uint8[:, :, 2], 3)  # B

print(f"STEP 6. QGIS용 RGB GeoTIFF 저장 완료: {OUT_PATH}")


STEP 0. 라이브러리 로드 완료
STEP 1. 파일 경로 설정 완료
STEP 2. 밴드 로드 함수 정의 완료
STEP 3. 구름 마스크 생성 완료 | cloud pixel 수 = 157187
STEP 4. RGB 밴드 로드 및 구름 마스크 적용 완료


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_32068\3414038307.py:76: RuntimeWarning: invalid value encountered in cast
  rgb8 = (rgb_preview * 255).astype(np.uint8)


True